## Practice: Data pipeline Hands-on Lab

Goal: Understand ETL logic.

We would need first to extract data from a MongoDB database comtaining customer reviews.

In [2]:
#pip install mongomock

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 2.6 MB/s eta 0:00:00


## Connect to an in-memory MongoDB instance

---



In [3]:
import mongomock

# The code below create a simulated MongoDB client and database
client = mongomock.MongoClient() #not a real server for practicality and IT restrictions (stored in memory)
db = client.my_mock_database # database creation
review_collection = db.reviews # collection creation (collections are like tables in SQL)

## Extract data from MongoDB


In [5]:
JSONL_path = "product_reviews_sales.jsonl"

| **Column**      | **Type**           | **Description**                                                                | **Example**                                           |
| --------------- | ------------------ | ------------------------------------------------------------------------------ | ----------------------------------------------------- |
| `product_id`    | `int`              | Unique product ID                  | `3`                                                   |
| `customer_id`   | `str`              | Unique identifier for the customer posting the review or making the purchase   | `"CUST_1042"`                                         |
| `review_text`   | `str`              | Free-text product review written by a customer                                 | `"Great quality and fits perfectly! Will buy again."` |
| `sale_date`     | `str` (ISO format) | Date of sale or review submission (YYYY-MM-DD)                                 | `"2024-11-09"`                                        |
| `units_sold`    | `int`              | Quantity sold during this transaction or review context                        | `2`                                                   |

In [11]:
import json

# Load and parse all lines from the JSONL file
with open(JSONL_path, "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f]
    print(data)

[{'product_id': 1, 'customer_id': 'C101', 'review_text': 'Great quality backpack, fits my laptop perfectly. Could use more pockets.', 'sale_date': '2024-10-15', 'units_sold': 3}, {'product_id': 1, 'customer_id': 'C102', 'review_text': 'Good for daily use but straps feel a bit thin.', 'sale_date': '2024-10-17', 'units_sold': 1}, {'product_id': 2, 'customer_id': 'C103', 'review_text': 'Nice shirt, fits well and comfortable fabric.', 'sale_date': '2024-09-20', 'units_sold': 2}, {'product_id': 3, 'customer_id': 'C104', 'review_text': 'Excellent jacket for winter hikes, very warm!', 'sale_date': '2024-11-02', 'units_sold': 1}, {'product_id': 4, 'customer_id': 'C105', 'review_text': 'Size runs small. Material is okay for the price.', 'sale_date': '2024-09-28', 'units_sold': 2}, {'product_id': 5, 'customer_id': 'C106', 'review_text': 'Beautiful bracelet, gift for my partner and she loved it!', 'sale_date': '2024-10-10', 'units_sold': 1}, {'product_id': 6, 'customer_id': 'C107', 'review_text':

In [12]:
#Insert all records into the collection using .insert_many()
result = review_collection.insert_many(data)

#count how many documents have been inserted
print(f"Inserted {len(result.inserted_ids)} documents into 'reviews' collection.")

Inserted 21 documents into 'reviews' collection.


In [13]:
#Let's check columns from data
sample = review_collection.find_one()
print("Fields in one document:", list(sample.keys()))

Fields in one document: ['product_id', 'customer_id', 'review_text', 'sale_date', 'units_sold', '_id']


In [14]:
import pandas as pd

df_reviews = pd.DataFrame((review_collection.find({})))
df_reviews.head()

,product_id,customer_id,review_text,sale_date,units_sold,_id
0,1,C101,"Great quality backpack, fits my laptop perfect...",2024-10-15,3,503b531e-c15e-11f0-a0d5-0242ac1c000c
1,1,C102,Good for daily use but straps feel a bit thin.,2024-10-17,1,503b5b34-c15e-11f0-a0d5-0242ac1c000c
2,2,C103,"Nice shirt, fits well and comfortable fabric.",2024-09-20,2,503b5de6-c15e-11f0-a0d5-0242ac1c000c
3,3,C104,"Excellent jacket for winter hikes, very warm!",2024-11-02,1,503b6020-c15e-11f0-a0d5-0242ac1c000c
4,4,C105,Size runs small. Material is okay for the price.,2024-09-28,2,503b61e2-c15e-11f0-a0d5-0242ac1c000c


## Extract data from an API


In [16]:
API_URL = "https://fakestoreapi.com/products"


The Fake Store API is a free, public REST API that provides mock e-commerce data for learning and testing.
A REST API (Representational State Transfer API) is a style of web API that lets clients interact with a server using HTTP requests.
Most REST APIs transfer data in JSON.

Fetch JSON data

In [18]:
import requests #popular Python library for making HTTP requests (GET, POST, PUT, DELETE, etc.).

#We send a GET request to the API endpoint to retrieve product data in JSON format:

resp = requests.get(API_URL)   # simple GET, no auth
products_json = resp.json()    # list of product
print(len(products_json), products_json[0])  # quick peek (count + first item)


20 {'id': 1, 'title': 'Fjallraven - Foldsack No. 1 Backpack, Fits 15 Laptops', 'price': 109.95, 'description': 'Your perfect pack for everyday use and walks in the forest. Stash your laptop (up to 15 inches) in the padded sleeve, your everyday', 'category': "men's clothing", 'image': 'https://fakestoreapi.com/img/81fPKd-2AYL._AC_SL1500_t.png', 'rating': {'rate': 3.9, 'count': 120}}


In [25]:
#transform json data into a dataframe
products_df = pd.DataFrame(products_json)
products_df.head()


,id,title,price,description,category,image,rating
0,1,"Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...",109.95,Your perfect pack for everyday use and walks i...,men's clothing,https://fakestoreapi.com/img/81fPKd-2AYL._AC_S...,"{'rate': 3.9, 'count': 120}"
1,2,Mens Casual Premium Slim Fit T-Shirts,22.30,"Slim-fitting style, contrast raglan long sleev...",men's clothing,https://fakestoreapi.com/img/71-3HjGNDUL._AC_S...,"{'rate': 4.1, 'count': 259}"
2,3,Mens Cotton Jacket,55.99,great outerwear jackets for Spring/Autumn/Wint...,men's clothing,https://fakestoreapi.com/img/71li-ujtlUL._AC_U...,"{'rate': 4.7, 'count': 500}"
3,4,Mens Casual Slim Fit,15.99,The color could be slightly different between ...,men's clothing,https://fakestoreapi.com/img/71YXzeOuslL._AC_U...,"{'rate': 2.1, 'count': 430}"
4,5,John Hardy Women's Legends Naga Gold & Silver ...,695.00,"From our Legends Collection, the Naga was insp...",jewelery,https://fakestoreapi.com/img/71pWzhdJNwL._AC_U...,"{'rate': 4.6, 'count': 400}"


In [26]:
# Flatten the nested 'rating' dictionary
rating_df = pd.json_normalize(products_df['rating'])

In [27]:
rating_df.head()

,rate,count
0,3.9,120
1,4.1,259
2,4.7,500
3,2.1,430
4,4.6,400


In [28]:
products_df= products_df.drop(columns=['rating']).join(rating_df)

In [29]:
products_df.head()

,id,title,price,description,category,image,rate,count
0,1,"Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...",109.95,Your perfect pack for everyday use and walks i...,men's clothing,https://fakestoreapi.com/img/81fPKd-2AYL._AC_S...,3.9,120
1,2,Mens Casual Premium Slim Fit T-Shirts,22.30,"Slim-fitting style, contrast raglan long sleev...",men's clothing,https://fakestoreapi.com/img/71-3HjGNDUL._AC_S...,4.1,259
2,3,Mens Cotton Jacket,55.99,great outerwear jackets for Spring/Autumn/Wint...,men's clothing,https://fakestoreapi.com/img/71li-ujtlUL._AC_U...,4.7,500
3,4,Mens Casual Slim Fit,15.99,The color could be slightly different between ...,men's clothing,https://fakestoreapi.com/img/71YXzeOuslL._AC_U...,2.1,430
4,5,John Hardy Women's Legends Naga Gold & Silver ...,695.00,"From our Legends Collection, the Naga was insp...",jewelery,https://fakestoreapi.com/img/71pWzhdJNwL._AC_U...,4.6,400


### Data transformation: MERGE DATA


We are going to merge both dataframes/combined data from 2 different sources.

In [33]:
combined_df = df_reviews.merge(
    products_df,
    left_on="product_id",
    right_on="id",
    how="left"         # keep all reviews, even if product missing
)
combined_df.head()


,product_id,customer_id,review_text,sale_date,units_sold,_id,id,title,price,description,category,image,rate,count
0,1,C101,"Great quality backpack, fits my laptop perfect...",2024-10-15,3,503b531e-c15e-11f0-a0d5-0242ac1c000c,1,"Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...",109.95,Your perfect pack for everyday use and walks i...,men's clothing,https://fakestoreapi.com/img/81fPKd-2AYL._AC_S...,3.9,120
1,1,C102,Good for daily use but straps feel a bit thin.,2024-10-17,1,503b5b34-c15e-11f0-a0d5-0242ac1c000c,1,"Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...",109.95,Your perfect pack for everyday use and walks i...,men's clothing,https://fakestoreapi.com/img/81fPKd-2AYL._AC_S...,3.9,120
2,2,C103,"Nice shirt, fits well and comfortable fabric.",2024-09-20,2,503b5de6-c15e-11f0-a0d5-0242ac1c000c,2,Mens Casual Premium Slim Fit T-Shirts,22.30,"Slim-fitting style, contrast raglan long sleev...",men's clothing,https://fakestoreapi.com/img/71-3HjGNDUL._AC_S...,4.1,259
3,3,C104,"Excellent jacket for winter hikes, very warm!",2024-11-02,1,503b6020-c15e-11f0-a0d5-0242ac1c000c,3,Mens Cotton Jacket,55.99,great outerwear jackets for Spring/Autumn/Wint...,men's clothing,https://fakestoreapi.com/img/71li-ujtlUL._AC_U...,4.7,500
4,4,C105,Size runs small. Material is okay for the price.,2024-09-28,2,503b61e2-c15e-11f0-a0d5-0242ac1c000c,4,Mens Casual Slim Fit,15.99,The color could be slightly different between ...,men's clothing,https://fakestoreapi.com/img/71YXzeOuslL._AC_U...,2.1,430


A left join means:

Keep all rows from the left DataFrame (df_reviews)

Only bring product info if a match exists

If no matching product exists → fill with NaN


In real-world ETL, not all data sources align perfectly.

Some reviews may refer to missing or discontinued products.

A left join preserves your operational data (reviews) while flagging missing lookups for investigation or logging.

Why NOT use a full outer join in this case?

A full outer join keeps everything from both tables, even when there’s no match:

All reviews (even if the product is missing)

All products (even if nobody reviewed them)

But in a reviews → products lookup scenario, this usually adds noise, not value.

Let's save our combined dataset:

In [34]:
combined_df.to_csv("combined_df.csv")